# Ejercicio

Los parámetros de un criptosistema de ElGamal son $p = 211$ y $g = 3$; es decir, el criptosistema está diseñado en el cuerpo $\mathbb{F}_{211} = \mathbb{Z}_{211}$ y tomamos como generador de $\mathbb{F}^*_{211}, g = 3$. La clave pública empleada es $3^\alpha = 109 \bmod 211$. 

Descifra el criptograma $(154, \text{dni} \bmod 211)$, donde $\text{dni}$ es el número de tu DNI. 

Para calcular los logaritmos discretos necesarios, emplea dos de los métodos descritos en la teoría.

## Preámbulos

In [5]:
p = 211 
g = 3

En este caso, como mi DNI es 

In [6]:
dni = 77432071

Aplicando módulo, se tiene

In [7]:
dni = dni.mod(211)
dni

135

Así que tenemos que descifrar el criptograma $(154, 135)$

## Algoritmo 1: Paso de bebé - Paso de gigante

Necesitamos descifrar el mensaje mediante $D_\alpha(x, y) = y \cdot x^{-\alpha}$. Para ello, vamos a calcular $\alpha$ mediante el logaritmo discreto. 

Por hipótesis, la clave pública es $3^\alpha = 109 \bmod 211$, por lo que 

$$
\alpha = \log_3{109} \bmod 211
$$

Se tiene que $G = \mathbb{F}^*_{211}, g = 3, p = 211, h = 109$ y que 

$$
f = \left \lceil \sqrt{p - 1} \right \rceil = \left \lceil \sqrt{210} \right \rceil = 15
$$

In [8]:
f = ceil(sqrt(p - 1))
f

15

Preparamos la tabla, donde cada entrada i-ésima es $g^i \bmod p$:

In [9]:
tabla = [[i, power_mod(g, i, p)] for i in range(0, f)]
tabla

[[0, 1],
 [1, 3],
 [2, 9],
 [3, 27],
 [4, 81],
 [5, 32],
 [6, 96],
 [7, 77],
 [8, 20],
 [9, 60],
 [10, 180],
 [11, 118],
 [12, 143],
 [13, 7],
 [14, 21]]

El algoritmo opera de la siguiente manera:

$$
g^{-f} = 3^{-15} = 3^{211 - 1 - 15} = 3^{195} = 67 \bmod 211
$$

Tenemos que $h_0$ = 109 no está en la tabla:

In [10]:
def esta_en_tabla(h, tabla):
    matches = [x for x in tabla if x[1] == h]
    
    return len(matches) > 0

In [11]:
esta_en_tabla(109, tabla)

False

Así que ahora debemos ir construyendo $h_i = h_{i-1} g^{-f}$:

In [12]:
h = 109
g_inv_f = power_mod(g,p-1-f, p)

print("h_0 = " + str(h))

indice = 0

for i in range(1, len(tabla)):
    h = (h * g_inv_f).mod(p)

    print("h_"+ str(i) + " = " + str(h))
    
    if esta_en_tabla(h, tabla):
        print("Sí que está en la tabla")
        indice = i
        break

h_0 = 109
h_1 = 129
h_2 = 203
h_3 = 97
h_4 = 169
h_5 = 140
h_6 = 96
Sí que está en la tabla


Por tanto, $\alpha$ vale 

In [13]:
alpha = indice + indice * f
alpha

96

$$
\begin{aligned}
\alpha & = \log_3{109} = 6 + 6 \cdot 15 = \\ 
       & = 96
\end{aligned}
$$

Así que el mensaje es

$$
D_{96}(154, dni) = D_{96}(154, 135) = 
$$

In [14]:
(dni * power_mod(154, p - 1 - alpha, p)).mod(p)

198

## Algoritmo 2: Silver - Pohlig - Hellman

Como con el algoritmo anterior, consideramos $p = 211, h = 109$, y un generador de $\mathbb{F}^{*}_{211}, g = 3$. 

Sabemos que $n = \text{orden}(\mathbb{F}^{*}_{211}) = p - 1 = 210$

In [15]:
n = p - 1

Los factores de $n$ son

In [90]:
factor(n)

2 * 3 * 5 * 7

In [91]:
h = 109

In [92]:
def silver_pohlig_hellman(b, h, p):
    n = p - 1
    factores_n = list(factor(n))
    
    print("Lista de factores de n = " + str(n) + ": " + str(factores_n))
    
    for _factor in factores_n:
        p_i = _factor[0]
        e_i = _factor[1]
        
        print("Para p_i = " + str(p_i) + ", obtenemos")
        r = [power_mod(b, (j * n)//p_i, p) for j in range(0, p_i)]
        
        print("  r = " + str(r))
        
        # Inicializar correctamente x e y 
        y = [h]
        x = []
        
        for j in range(len(r)):
            if power_mod(y[0], n // p_i, p) == r[j]: 
                x.append(j)

        print("  y_0^(n / p_i) = " + str(y[0]) + "^" + str(n) + "/" + str(p_i) + "= " + str(power_mod(y[0], n // p_i, p)))
        print("  x = " + str(x) + ", y = " + str(y))
        
        for k in range(0, e_i):
            break
            # En este caso, todos los índices son 1, así que voy a hacer la 
            # guarrería de dejar este bucle vacío. 
            # Si esto fuera un algoritmo hecho y derecho, haría falta completar 
            # este bucle 
            
        print()

In [93]:
silver_pohlig_hellman(g, h, p)

Lista de factores de n = 210: [(2, 1), (3, 1), (5, 1), (7, 1)]
Para p_i = 2, obtenemos
  r = [1, 210]
  y_0^(n / p_i) = 109^210/2= 1
  x = [0], y = [109]

Para p_i = 3, obtenemos
  r = [1, 196, 14]
  y_0^(n / p_i) = 109^210/3= 1
  x = [0], y = [109]

Para p_i = 5, obtenemos
  r = [1, 188, 107, 71, 55]
  y_0^(n / p_i) = 109^210/5= 188
  x = [1], y = [109]

Para p_i = 7, obtenemos
  r = [1, 171, 123, 144, 148, 199, 58]
  y_0^(n / p_i) = 109^210/7= 199
  x = [5], y = [109]

